In [1]:
from pathlib import Path
import nibabel as nib
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("/home/jovyan/P1-Temp-Reg/notebooks")

import mathplus_p1_auxiliary_functions as aux

root_path = Path("/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/")

fpath_output = Path("/home/jovyan/InputData/")
fpath_output.mkdir(exist_ok=True, parents=True)


In [2]:
def get_subdirs(path: Path):
    list_paths = []
    for path in Path(path).iterdir():
        if path.is_dir():
            list_paths.append(path)
    
    return list_paths

dirs_acdc = get_subdirs(root_path)


In [3]:
def crop_image(imgarr, cropped_size):
    
    imgdims = np.array(imgarr.shape)//2
    xstart = imgdims[0] - cropped_size//2
    xend = imgdims[0] + cropped_size//2
    
    ystart = imgdims[1] - cropped_size//2
    yend = imgdims[1] + cropped_size//2
    

    return imgarr[xstart:xend, ystart:yend,...]

def get_cropped_image(path: Path):
    
    img = nib.load(str(path / "image.nii.gz"))

    cropsize=128
    cropped_arr = crop_image(img.get_fdata(), cropsize)

    return np.squeeze(cropped_arr)

def get_formatted_image(path: Path, num_phases, time_usf):
    img = get_cropped_image(path)

    if img.shape[3] < num_phases:
        return None

    img = img[...,1:num_phases:time_usf]
    
    return img



In [4]:
min_num_phases = 24
time_usf = 2

i=0
for curr_dir in tqdm(dirs_acdc):
    list_patients = get_subdirs(curr_dir)
    
    for pat in list_patients:
        
        img = get_formatted_image(pat, min_num_phases, time_usf)
        if img is None:
            break
            
        for islice in range(img.shape[2]):
            dat = np.squeeze(img[:,:,islice,:])
            
            dat = np.swapaxes(dat, 0, 2)
            nii = nib.Nifti1Image(dat, np.eye(4))
            
            nib.save(nii, str(fpath_output / "img_{}.nii".format(i)))
            i += 1
        

100%|██████████| 4/4 [00:10<00:00,  2.73s/it]


Now we have to prepare the k-space data

In [5]:
import sirf.Gadgetron as pMR
import numpy as np


fname_template = fpath_output / "template/cine_128_30ph_bc.h5"
template_data = pMR.AcquisitionData(str(fname_template))

template_data = pMR.preprocess_acquisition_data(template_data)
template_phase = template_data.get_ISMRMRD_info("phase")

index_all_acquis = np.arange(template_data.number())

index_interesting_acquis = template_phase < (min_num_phases// time_usf)
index_interesting_acquis = index_all_acquis[index_interesting_acquis]
template_data = template_data.get_subset(index_interesting_acquis)

template_data = aux.undersample_cartesian_data(template_data)
print(template_data.shape)

Started reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..100%..
Finished reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5
Nky 128 - Ncph 12
(384, 1, 128)


In [9]:
import sirf.Reg as pReg

csm = pMR.CoilSensitivityData()
csm.calculate(template_data)
csm.fill(1 + 0. * 1j)

template_img = pMR.ImageData()
template_img.from_acquisition_data(template_data)

E = pMR.AcquisitionModel(acqs=template_data, imgs=template_img)
E.set_coil_sensitivity_maps(csm)

template_fwd = pMR.ImageData()
template_fwd.from_acquisition_data(template_data)

path_input = fpath_output
list_stacks = sorted(path_input.glob("img*"))


In [ ]:


for f in tqdm(list_stacks):
        
    fileidx = str(f.stem).split('_')[1]
        
    nii = pReg.ImageData(str(f))
    template_fwd.fill(nii.as_array())
    
    rd = E.forward(template_fwd)
    
    fout = fpath_output / "y_{}.h5".format(fileidx)
    rd.write(str(fout))
    

In [14]:

list_rawdata = sorted(path_input.glob("y_*"))

for f in tqdm(list_rawdata):
        
    fileidx = str(f.stem).split('_')[1]
    ytmp = pMR.AcquisitionData(str(f))
    
    itmp = E.backward(ytmp)
    
    nii = nib.Nifti1Image(itmp.as_array(), np.eye(4))
    fout = fpath_output / "cmplx_zfrecon_{}.nii".format(fileidx)
    nib.save(nii,str(fout))
    
    
    nii = nib.Nifti1Image(np.abs(itmp.as_array()), np.eye(4))
    fout = fpath_output / "zfrecon_{}.nii".format(fileidx)
    nib.save(nii,str(fout))
    
    

  0%|          | 1/325 [00:00<01:25,  3.80it/s]

Started reading acquisitions from /home/jovyan/InputData/y_0.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_0.h5


  1%|          | 2/325 [00:00<01:17,  4.19it/s]

Started reading acquisitions from /home/jovyan/InputData/y_1.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_1.h5


  1%|          | 4/325 [00:00<01:05,  4.93it/s]

...

Started reading acquisitions from /home/jovyan/InputData/y_94.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_94.h5
Started reading acquisitions from /home/jovyan/InputData/y_95.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_95.h5
Started reading acquisitions from /home/jovyan/InputData/y_96.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_96.h5


100%|█████████▉| 324/325 [00:50<00:00,  6.52it/s]

Started reading acquisitions from /home/jovyan/InputData/y_97.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_97.h5
Started reading acquisitions from /home/jovyan/InputData/y_98.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_98.h5


100%|██████████| 325/325 [00:50<00:00,  6.41it/s]

Started reading acquisitions from /home/jovyan/InputData/y_99.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..99%..
Finished reading acquisitions from /home/jovyan/InputData/y_99.h5
